# Intrusion Detection using LightGBM

## Nitin Vankadari

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
%matplotlib inline

In [2]:
dataset=pd.read_csv("train_Test_Linux_process.csv")

In [3]:
dataset.head()

,ts,PID,TRUN,TSLPI,TSLPU,POLI,NICE,PRI,RTPR,CPUNR,Status,EXC,State,CPU,CMD,label,type
0,1556129658,52888,0,1,0,norm,20,100,0,0,0,0,S,0.01,atop,1,dos
1,1556129738,2791,0,5,0,norm,0,120,0,0,0,0,S,0.00,nautilus,1,dos
2,1556129778,2504,0,1,0,norm,0,120,0,2,0,0,S,0.00,upstart-dbus-b,1,dos
3,1556129788,3147,1,12,0,norm,0,120,0,1,0,0,S,1.00,drone,1,dos
4,1556129798,52888,0,1,0,norm,20,100,0,0,0,0,S,0.01,atop,1,dos


In [4]:
dataset.shape

(160112, 17)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160112 entries, 0 to 160111
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ts      160112 non-null  int64  
 1   PID     160112 non-null  int64  
 2   TRUN    160112 non-null  int64  
 3   TSLPI   160112 non-null  int64  
 4   TSLPU   160112 non-null  int64  
 5   POLI    160112 non-null  object 
 6   NICE    160112 non-null  int64  
 7   PRI     160112 non-null  int64  
 8   RTPR    160112 non-null  int64  
 9   CPUNR   160112 non-null  int64  
 10  Status  160112 non-null  object 
 11  EXC     160112 non-null  int64  
 12  State   160112 non-null  object 
 13  CPU     160112 non-null  float64
 14  CMD     160112 non-null  object 
 15  label   160112 non-null  int64  
 16  type    160112 non-null  object 
dtypes: float64(1), int64(11), object(5)
memory usage: 20.8+ MB


In [6]:
dataset.describe()

,ts,PID,TRUN,TSLPI,TSLPU,NICE,PRI,RTPR,CPUNR,EXC,CPU,label
count,1.601120e+05,160112.000000,160112.000000,160112.000000,160112.000000,160112.000000,160112.000000,160112.0,160112.000000,160112.000000,160112.000000,160112.000000
mean,1.555145e+09,3256.037293,0.083092,3.916814,0.001130,5.157471,111.654748,0.0,1.569351,0.017663,0.038341,0.375437
std,8.816813e+05,2071.338868,0.284315,7.856580,0.074001,8.722292,20.521090,0.0,1.245371,0.692833,0.188411,0.484237
min,1.554219e+09,1007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,1.554419e+09,2533.000000,0.000000,1.000000,0.000000,0.000000,100.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,1.554619e+09,3058.000000,0.000000,1.000000,0.000000,0.000000,120.000000,0.0,2.000000,0.000000,0.000000,0.000000
75%,1.556223e+09,3793.000000,0.000000,4.000000,0.000000,20.000000,120.000000,0.0,3.000000,0.000000,0.010000,1.000000
max,1.556549e+09,53075.000000,9.000000,67.000000,21.000000,20.000000,139.000000,0.0,5.000000,100.000000,4.000000,1.000000


In [7]:
dataset.isnull().sum()

ts        0
PID       0
TRUN      0
TSLPI     0
TSLPU     0
POLI      0
NICE      0
PRI       0
RTPR      0
CPUNR     0
Status    0
EXC       0
State     0
CPU       0
CMD       0
label     0
type      0
dtype: int64

In [8]:
dataset['CMD'].value_counts()

atop              40831
vmtoolsd           9304
apache2            8347
Xorg               5873
nautilus           5665
                  ...  
<JS Helper>           1
<gpg>                 1
kworker/5:5+ev        1
<udev-configu>        1
<apt-key>             1
Name: CMD, Length: 406, dtype: int64

In [9]:
dataset['label'].value_counts()

0    100000
1     60112
Name: label, dtype: int64

In [10]:
dataset['type'].value_counts()

normal       100000
password      10000
xss           10000
dos           10000
injection     10000
ddos          10000
scanning      10000
mitm            112
Name: type, dtype: int64

In [11]:
dataset['State'].value_counts()

S    145383
R      8583
E      4315
I      1590
D       129
Z       112
Name: State, dtype: int64

In [12]:
dataset['Status'].value_counts()

-     125604
0      29080
N       2759
NE      2620
NS        47
NC         1
C          1
Name: Status, dtype: int64

In [13]:
dataset['POLI'].value_counts()

norm    155797
0         2910
-         1405
Name: POLI, dtype: int64

In [14]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
dataset["_CMD_"]=LE.fit_transform(dataset.CMD)
dataset["_type_"]=LE.fit_transform(dataset.type)
dataset["_POLI_"]=LE.fit_transform(dataset.POLI)
dataset["_State_"]=LE.fit_transform(dataset.State)
dataset["_Status_"]=LE.fit_transform(dataset.Status)
dataset.drop(['CMD','type','POLI','Status','State'],axis=1,inplace=True)

In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160112 entries, 0 to 160111
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ts        160112 non-null  int64  
 1   PID       160112 non-null  int64  
 2   TRUN      160112 non-null  int64  
 3   TSLPI     160112 non-null  int64  
 4   TSLPU     160112 non-null  int64  
 5   NICE      160112 non-null  int64  
 6   PRI       160112 non-null  int64  
 7   RTPR      160112 non-null  int64  
 8   CPUNR     160112 non-null  int64  
 9   EXC       160112 non-null  int64  
 10  CPU       160112 non-null  float64
 11  label     160112 non-null  int64  
 12  _CMD_     160112 non-null  int32  
 13  _type_    160112 non-null  int32  
 14  _POLI_    160112 non-null  int32  
 15  _State_   160112 non-null  int32  
 16  _Status_  160112 non-null  int32  
dtypes: float64(1), int32(5), int64(11)
memory usage: 17.7 MB


In [16]:
dataset.head()

,ts,PID,TRUN,TSLPI,TSLPU,NICE,PRI,RTPR,CPUNR,EXC,CPU,label,_CMD_,_type_,_POLI_,_State_,_Status_
0,1556129658,52888,0,1,0,20,100,0,0,0,0.01,1,166,1,2,4,1
1,1556129738,2791,0,5,0,0,120,0,0,0,0.00,1,345,1,2,4,1
2,1556129778,2504,0,1,0,0,120,0,2,0,0.00,1,391,1,2,4,1
3,1556129788,3147,1,12,0,0,120,0,1,0,1.00,1,188,1,2,4,1
4,1556129798,52888,0,1,0,20,100,0,0,0,0.01,1,166,1,2,4,1


In [17]:
dataset["_type_"].value_counts()

4    100000
7     10000
6     10000
5     10000
2     10000
1     10000
0     10000
3       112
Name: _type_, dtype: int64

In [18]:
y=dataset['_type_']

In [19]:
x=dataset.drop(['_type_'], axis=1)

In [20]:
from sklearn.preprocessing import Normalizer
n_scalar = Normalizer()
xdataset = dataset.iloc[:,:-1]
col_names = xdataset.columns
normdataset = n_scalar.fit_transform(dataset.drop('_type_',1))
dataset = pd.DataFrame(normdataset, columns = col_names)

In [21]:
dataset.head()

,ts,PID,TRUN,TSLPI,TSLPU,NICE,PRI,RTPR,CPUNR,EXC,CPU,label,_CMD_,_type_,_POLI_,_State_
0,1.0,0.000034,0.000000e+00,6.426200e-10,0.0,1.285240e-08,6.426200e-08,0.0,0.000000e+00,0.0,6.426200e-12,6.426200e-10,1.066749e-07,1.285240e-09,2.570480e-09,6.426200e-10
1,1.0,0.000002,0.000000e+00,3.213100e-09,0.0,0.000000e+00,7.711439e-08,0.0,0.000000e+00,0.0,0.000000e+00,6.426199e-10,2.217039e-07,1.285240e-09,2.570480e-09,6.426199e-10
2,1.0,0.000002,0.000000e+00,6.426199e-10,0.0,0.000000e+00,7.711439e-08,0.0,1.285240e-09,0.0,0.000000e+00,6.426199e-10,2.512644e-07,1.285240e-09,2.570480e-09,6.426199e-10
3,1.0,0.000002,6.426199e-10,7.711439e-09,0.0,0.000000e+00,7.711439e-08,0.0,6.426199e-10,0.0,6.426199e-10,6.426199e-10,1.208125e-07,1.285240e-09,2.570480e-09,6.426199e-10
4,1.0,0.000034,0.000000e+00,6.426199e-10,0.0,1.285240e-08,6.426199e-08,0.0,0.000000e+00,0.0,6.426199e-12,6.426199e-10,1.066749e-07,1.285240e-09,2.570480e-09,6.426199e-10


In [22]:
y.value_counts()

4    100000
7     10000
6     10000
5     10000
2     10000
1     10000
0     10000
3       112
Name: _type_, dtype: int64

In [23]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
s_strategy = {0: 10000, 1: 10000, 2:10000, 3:10000, 4:100000, 5:10000, 6: 10000, 7: 10000}
ro=RandomOverSampler(sampling_strategy=s_strategy,random_state=1)
Xres1,yres1=ro.fit_resample(dataset,y)

In [24]:
yres1.value_counts()

4    100000
7     10000
6     10000
5     10000
3     10000
2     10000
1     10000
0     10000
Name: _type_, dtype: int64

In [25]:
from imblearn.under_sampling import RandomUnderSampler
s_strategy = {0: 10000, 1: 10000, 2:10000, 3:10000, 4:10000, 5:10000, 6: 10000, 7: 10000}
ro=RandomUnderSampler(sampling_strategy=s_strategy,random_state=1)
Xr1,yr1=ro.fit_resample(Xres1,yres1)

In [26]:
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold,KFold,cross_val_score
from numpy import mean

In [27]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Xr1,yr1, test_size=0.25, random_state=42, stratify=yr1)

In [28]:
import lightgbm as lgbm
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=42)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9523

In [30]:
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from numpy import mean
model = LGBMClassifier(random_state=42)
def evaluate_model(model):
    K_Fold = KFold(n_splits=10)
    score_of_kfold = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=K_Fold, n_jobs=-1, verbose = 1, error_score='raise')
    cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=42)
    score_of_stratified = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, verbose = 1, error_score='raise')
    list_of_scores = [mean(score_of_kfold),mean(score_of_stratified)]
    return list_of_scores
final_scores = (evaluate_model(model))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   56.1s remaining:   37.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   24.9s remaining:   16.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   32.6s finished


In [31]:
final_scores

[0.9489333333333334, 0.9494666666666667]

In [32]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics

In [33]:
model.fit(X_train, y_train)
pred1=model.predict(X_test)

In [34]:
confusion_matrix(y_test,pred1)

array([[2496,    4,    0,    0,    0,    0,    0,    0],
       [   6, 2298,  196,    0,    0,    0,    0,    0],
       [   0,  744, 1756,    0,    0,    0,    0,    0],
       [   0,    0,    0, 2500,    0,    0,    0,    0],
       [   0,    0,    0,    0, 2500,    0,    0,    0],
       [   0,    0,    0,    0,    0, 2498,    0,    2],
       [   1,    0,    0,    0,    0,    0, 2499,    0],
       [   0,    0,    0,    0,    0,    1,    0, 2499]], dtype=int64)

In [35]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
def get_classification_report(y_test, y_pred):
    from sklearn import metrics
    report = metrics.classification_report(y_test, y_pred, output_dict=True)
    df_classification_report = pd.DataFrame(report).transpose()
    df_classification_report = df_classification_report.sort_values(by=['f1-score'], ascending=False)
    return df_classification_report
get_classification_report(y_test, y_pred)

,precision,recall,f1-score,support
3,1.000000,1.0000,1.000000,2500.0000
4,1.000000,1.0000,1.000000,2500.0000
6,1.000000,0.9996,0.999800,2500.0000
7,0.999200,0.9996,0.999400,2500.0000
5,0.999600,0.9992,0.999400,2500.0000
0,0.997203,0.9984,0.997801,2500.0000
accuracy,0.952300,0.9523,0.952300,0.9523
macro avg,0.956253,0.9523,0.951746,20000.0000
weighted avg,0.956253,0.9523,0.951746,20000.0000
1,0.754432,0.9192,0.828705,2500.0000


In [28]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.92065

In [29]:
from sklearn.model_selection import StratifiedKFold,KFold,RepeatedStratifiedKFold,cross_val_score
model = XGBClassifier(random_state=42)
def evaluate_model(model):
    KF=KFold(n_splits=10)
    score1 = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=KF)
    list_scores=[mean(score1)]
    return list_scores
scores = evaluate_model(model)
names=["KFold"]
print("KFOLD CROSS VALIDATION SCORE")
print("------------------------------------")
for (i,j) in zip(scores,names):
    print(j,"-",round(i*100,2),'%')

KFOLD CROSS VALIDATION SCORE
------------------------------------
KFold - 91.81 %


In [29]:
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold,KFold,RepeatedStratifiedKFold,cross_val_score
from xgboost import XGBClassifier
from numpy import mean

In [30]:
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.92065

In [31]:
model = XGBClassifier(random_state=42)
def evaluate_model(model):    
    KF=KFold(n_splits=10)
    score1 = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=KF) 
    list_scores=[mean(score1)]
    return list_scores
scores = evaluate_model(model)

In [32]:
names=["KFold"]
print("KFOLD CROSS VALIDATION SCORE")
print("------------------------------------")
for (i,j) in zip(scores,names):
    print(j,"-",round(i*100,2),'%')

KFOLD CROSS VALIDATION SCORE
------------------------------------
KFold - 91.81 %


In [33]:
params={'max_depth':[3], 'n_estimators': [250,400],'learning_rate': [0.01,0.1], 'subsample':[0.6,0.9]}

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
grid = GridSearchCV(XGBClassifier(),params,refit=True,verbose=3)

In [39]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6, score=0.855, total= 1.5min
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6, score=0.851, total= 1.5min
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6, score=0.857, total= 1.5min
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6 
[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6, score=0.852, total= 1.4min
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6 
[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.6, score=0.853, total= 1.4min
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.9 
[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.9, score=0.854, total= 1.2min
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.9 
[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.9, score=0.850, total= 1.2min
[CV] learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.9 
[CV]  learning_rate=0.01, max_depth=3, n_estimators=250, subsample=0.9, score=0.857, total= 1.2min
[CV] learning_rate=0.01, max_depth=3, n_estimators=

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 68.4min finished


GridSearchCV(estimator=XGBClassifier(),
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [3],
                         'n_estimators': [250, 400], 'subsample': [0.6, 0.9]},
             verbose=3)

In [40]:
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 400, 'subsample': 0.9}

In [41]:
grid.best_estimator_

XGBClassifier(n_estimators=400, objective='multi:softprob', subsample=0.9)

In [42]:
grid_predictions = grid.predict(X_test)

In [43]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,grid_predictions))

[[2486    8    6    0    0    0    0    0]
 [  12 2270  218    0    0    0    0    0]
 [   7  775 1718    0    0    0    0    0]
 [   0    0    0 2500    0    0    0    0]
 [   0    0    0    0 2500    0    0    0]
 [   0    0    0    0    0 2500    0    0]
 [   0    0    0    0    0    0 2500    0]
 [   0    0    0    0    0    4    0 2496]]


In [44]:
print(classification_report(y_test,grid_predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2500
           1       0.74      0.91      0.82      2500
           2       0.88      0.69      0.77      2500
           3       1.00      1.00      1.00      2500
           4       1.00      1.00      1.00      2500
           5       1.00      1.00      1.00      2500
           6       1.00      1.00      1.00      2500
           7       1.00      1.00      1.00      2500

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000

